In [1]:
import pandas as pd
import numpy as np

In [2]:
trocafone =  pd.read_csv("trocafone_kaggle_test.csv")
events = pd.read_csv("events_up_to_01062018.csv",dtype = {3: object,5: object,
                    6: object,7: object,
                    8: object,9: object,
                    10: object,11: object,
                    12: object,13: object,
                    14: object,15: object,
                    16: object,17: object,
                    18: object,19: object,
                    20: object,21: object,
                    22: object})
labels = pd.read_csv("labels_training_set.csv")

In [3]:
events.head()

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,search_engine,channel,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version
0,2018-05-18 00:11:59,viewed product,4886f805,NaN,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-05-18 00:11:27,viewed product,ad93850f,NaN,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-05-18 00:11:16,viewed product,0297fc1e,NaN,6888.0,iPhone 6S,Muito Bom,64GB,Prateado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-05-18 00:11:14,viewed product,2d681dd8,NaN,11890.0,iPhone 7,Bom,128GB,Vermelho,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-05-18 00:11:09,viewed product,cccea85e,NaN,7517.0,LG G4 H818P,Excelente,32GB,Branco,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# TRAIN

In [3]:
converted = pd.merge(events,labels,on="person",how="inner")

In [4]:
converted["timestamp"] = pd.to_datetime(converted["timestamp"])

In [6]:
def change_column_order(df, col_name, index):
    cols = df.columns.tolist()
    cols.remove(col_name)
    cols.insert(index, col_name)
    return df[cols]

## Features sobre visited site

In [123]:
Ft_visited_site = pd.concat([converted[converted["event"] == "visited site"].iloc[:,:3],\
                         converted[converted["event"] == "visited site"].iloc[:,14:]],axis =1)

In [124]:
Ft_visited_site.head()

,timestamp,event,person,channel,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version,label
60,2018-05-14 23:50:22,visited site,ad93850f,Organic,New,São Paulo,Sao Paulo,Brazil,Smartphone,360x640,Android 5.1.1,Chrome Mobile 66.0,0
61,2018-05-16 02:48:13,visited site,ad93850f,Paid,Returning,São Paulo,Sao Paulo,Brazil,Smartphone,360x640,Android 5.1.1,Chrome Mobile 66.0,0
62,2018-05-18 00:11:26,visited site,ad93850f,Paid,Returning,São Paulo,Sao Paulo,Brazil,Smartphone,360x640,Android 5.1.1,Chrome Mobile 66.0,0
63,2018-05-18 22:11:46,visited site,ad93850f,Paid,Returning,São Paulo,Sao Paulo,Brazil,Smartphone,360x640,Android 5.1.1,Chrome Mobile 66.0,0
64,2018-05-22 22:41:31,visited site,ad93850f,Paid,Returning,São Paulo,Sao Paulo,Brazil,Smartphone,360x640,Android 5.1.1,Chrome Mobile 66.0,0


####  countries populares

In [125]:
popular_countries = Ft_visited_site["country"].value_counts().nlargest(10).index

In [126]:
for country in popular_countries:
    Ft_visited_site[country] = (Ft_visited_site['country'] == country).astype(bool)

In [127]:
Ft_visited_site.drop(columns = "country",inplace = True)

#### Filtrar por channel

In [128]:
Ft_visited_site = Ft_visited_site[Ft_visited_site["channel"] != "Unknown"]

In [129]:
Ft_visited_site["channel"].value_counts()

Paid        45632
Direct      25593
Referral    17167
Organic     13050
Social        689
Email         167
Name: channel, dtype: int64

In [130]:
channels = Ft_visited_site["channel"].value_counts().index

In [131]:
for channel in channels:
    Ft_visited_site[channel] = (Ft_visited_site['channel'] == channel).astype(bool)

In [132]:
Ft_visited_site = Ft_visited_site.drop(columns = "channel")

####  browser



In [133]:
popular_browsers = Ft_visited_site["browser_version"].value_counts().nlargest(10).index

In [134]:
popular_browsers

Index(['Chrome 66.0', 'Chrome Mobile 66.0', 'Chrome Mobile 65.0',
       'Chrome 65.0', 'Chrome Mobile 64.0', 'Chrome 64.0', 'Mobile Safari 11',
       'Chrome Mobile 63.0', 'Chrome 63.0', 'Samsung Internet 6.4'],
      dtype='object')

In [135]:
for browser in popular_browsers:
    Ft_visited_site[browser] = (Ft_visited_site['browser_version'] == channel).astype(bool)

In [136]:
Ft_visited_site["browser_version"].shape

(102298,)

In [137]:
Ft_visited_site = Ft_visited_site.drop(columns = "browser_version")

#### Filtrar por nuevo vs recurrente

In [138]:
recurrencias = Ft_visited_site["new_vs_returning"].value_counts().index

In [139]:
for recurrencia in recurrencias:
    Ft_visited_site[recurrencia] = (Ft_visited_site['new_vs_returning'] == recurrencia).astype(bool)

In [140]:
Ft_visited_site = Ft_visited_site.drop(columns = ["city","region","new_vs_returning"])

#### screen_resolution

In [141]:
populars_resolutions = Ft_visited_site["screen_resolution"].value_counts().nlargest(10).index

In [142]:
for resolution in populars_resolutions:
    Ft_visited_site[resolution] = (Ft_visited_site['screen_resolution'] == resolution).astype(bool)

In [143]:
Ft_visited_site.head()

,timestamp,event,person,device_type,screen_resolution,operating_system_version,label,Brazil,Unknown,United States,...,360x640,1366x768,1920x1080,320x534,1600x900,1360x768,1440x900,320x570,1024x768,320x568
60,2018-05-14 23:50:22,visited site,ad93850f,Smartphone,360x640,Android 5.1.1,0,True,False,False,...,True,False,False,False,False,False,False,False,False,False
61,2018-05-16 02:48:13,visited site,ad93850f,Smartphone,360x640,Android 5.1.1,0,True,False,False,...,True,False,False,False,False,False,False,False,False,False
62,2018-05-18 00:11:26,visited site,ad93850f,Smartphone,360x640,Android 5.1.1,0,True,False,False,...,True,False,False,False,False,False,False,False,False,False
63,2018-05-18 22:11:46,visited site,ad93850f,Smartphone,360x640,Android 5.1.1,0,True,False,False,...,True,False,False,False,False,False,False,False,False,False
64,2018-05-22 22:41:31,visited site,ad93850f,Smartphone,360x640,Android 5.1.1,0,True,False,False,...,True,False,False,False,False,False,False,False,False,False


In [144]:
Ft_visited_site = Ft_visited_site.drop(columns = ["screen_resolution","operating_system_version"])

#### device_type

In [145]:
tipoDisp = Ft_visited_site["device_type"].value_counts().index

In [146]:
for dispositivo in tipoDisp:
    Ft_visited_site[dispositivo] = (Ft_visited_site['device_type'] == dispositivo).astype(bool)

In [147]:
Ft_visited_site = Ft_visited_site.drop(columns = ["device_type","event"])\
                    .rename(columns = {"Unknown" : "Unknown_country"})

In [148]:
Ft_visited_site["mes"] = Ft_visited_site["timestamp"].dt.month
Ft_visited_site.drop(columns = "timestamp", inplace =True)

In [149]:
Ft_visited_site.shape

(102298, 44)

In [151]:
Ft_visited_site = change_column_order(Ft_visited_site,"mes",0).drop(columns = "label")

In [152]:
Ft_visited_site = Ft_visited_site.sort_values(by="mes")


In [153]:
Ft_visited_site = Ft_visited_site.groupby(["mes","person"]).sum().reset_index()


In [154]:
Ft_visited_site.to_csv('FT_visited_site_train.csv',index = False)

## Features sobre viewed product


In [155]:
Ft_viewed_product = converted[converted["event"] == "viewed product"].iloc[:,0:9]\
                    .drop(columns = ["event","url"])

In [156]:
Ft_viewed_product["mes"] = Ft_viewed_product["timestamp"].dt.month

In [157]:
Ft_viewed_product.drop(columns = "timestamp",inplace = True)

In [158]:
Ft_viewed_product = change_column_order(Ft_viewed_product,"mes",0)

In [159]:
Ft_viewed_product = Ft_viewed_product.sort_values(by="mes")
Ft_viewed_product.head()

,mes,person,sku,model,condition,storage,color
649409,1,30ec9daf,3396.0,Samsung Galaxy Note Edge,Muito Bom,32GB,Preto
342604,1,6d4e29bb,2929.0,Motorola Moto MAXX,Excelente,64GB,Preto
342603,1,6d4e29bb,8485.0,Samsung Galaxy S7,Bom,32GB,Dourado
342602,1,6d4e29bb,10352.0,Motorola Moto G4 Plus,Excelente,32GB,Preto
342601,1,6d4e29bb,2976.0,Samsung Galaxy A7,Muito Bom,16GB,Branco


### model

In [160]:
modelos_populares = Ft_viewed_product["model"].value_counts().nlargest(15).index

In [161]:
modelos_populares = pd.Series(modelos_populares).values
modelos_populares

array(['iPhone 6', 'iPhone 5s', 'iPhone 6S', 'iPhone 7',
       'Samsung Galaxy S7 Edge', 'iPhone 7 Plus', 'Samsung Galaxy S7',
       'iPhone 6S Plus', 'iPhone SE', 'Samsung Galaxy S8',
       'Samsung Galaxy S6 Edge', 'iPhone 6 Plus', 'iPhone 5c',
       'Samsung Galaxy J5', 'Samsung Galaxy S6 Flat'], dtype=object)

In [162]:
for modelo in modelos_populares:
    Ft_viewed_product[modelo] = (Ft_viewed_product['model'] == modelo).astype(bool)

In [163]:
modelosView = pd.Series(modelos_populares).apply(lambda x: x +"_view").values
Ft_viewed_product.rename(columns=dict(zip(Ft_viewed_product.iloc[:,7:], modelosView)),inplace=True)

In [164]:
Ft_viewed_product.drop(columns = "model",inplace = True)
Ft_viewed_product.head()

,mes,person,sku,condition,storage,color,iPhone 6_view,iPhone 5s_view,iPhone 6S_view,iPhone 7_view,...,iPhone 7 Plus_view,Samsung Galaxy S7_view,iPhone 6S Plus_view,iPhone SE_view,Samsung Galaxy S8_view,Samsung Galaxy S6 Edge_view,iPhone 6 Plus_view,iPhone 5c_view,Samsung Galaxy J5_view,Samsung Galaxy S6 Flat_view
649409,1,30ec9daf,3396.0,Muito Bom,32GB,Preto,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
342604,1,6d4e29bb,2929.0,Excelente,64GB,Preto,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
342603,1,6d4e29bb,8485.0,Bom,32GB,Dourado,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
342602,1,6d4e29bb,10352.0,Excelente,32GB,Preto,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
342601,1,6d4e29bb,2976.0,Muito Bom,16GB,Branco,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


### colorView

In [165]:
#15 mas vistos
colores = Ft_viewed_product["color"].value_counts().nlargest(15).index
colores

Index(['Preto', 'Dourado', 'Cinza espacial', 'Branco', 'Prateado', 'Ouro Rosa',
       'Rosa', 'Prata', 'Preto Matte', 'Azul', 'Platinum', 'Preto Brilhante',
       'Ametista', 'Ouro', 'Vermelho'],
      dtype='object')

In [166]:
for color in colores:
    Ft_viewed_product[color] = (Ft_viewed_product['color'] == color).astype(bool)

In [167]:
Ft_viewed_product.drop(columns = "color",inplace = True)

In [168]:
Ft_viewed_product.drop(columns = "sku",inplace = True)

In [169]:
coloresView = pd.Series(colores).apply(lambda x: x +"_view").values
Ft_viewed_product.rename(columns=dict(zip(Ft_viewed_product.iloc[:,19:], coloresView)),inplace=True)

In [170]:
#En el tp1 no influyo demasiado en las conversiones el buen storage, los usuarios compraban modelos a bajo costo 
#sacrificando el almacenamiento, tampoco se puede decir que las conversiones se inclinan al almacenamiento bajo
#Si el dia de maniana sale un telefono con bajo costo y almacenamiento alto esto influria diferente en el algoritmo
#Lo descarto

In [171]:
Ft_viewed_product.drop(columns = "storage",inplace = True)

### condition

In [172]:
condiciones = Ft_viewed_product.condition.value_counts().index

In [173]:
for condition in condiciones:
    Ft_viewed_product[condition] = (Ft_viewed_product['condition'] == condition).astype(bool)

In [174]:
conditionView = pd.Series(condiciones).apply(lambda x: x +"_view").values
Ft_viewed_product.rename(columns=dict(zip(Ft_viewed_product.iloc[:,33:], conditionView)),inplace=True)

In [175]:
Ft_viewed_product.drop(columns = "condition",inplace = True)

In [176]:
Ft_viewed_product.head()


,mes,person,iPhone 6_view,iPhone 5s_view,iPhone 6S_view,iPhone 7_view,Samsung Galaxy S7 Edge_view,iPhone 7 Plus_view,Samsung Galaxy S7_view,iPhone 6S Plus_view,...,Platinum_view,Preto Brilhante_view,Ametista_view,Ouro_view,Vermelho_view,Bom_view,Excelente_view,Muito Bom_view,Bom - Sem Touch ID_view,Novo_view
649409,1,30ec9daf,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
342604,1,6d4e29bb,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
342603,1,6d4e29bb,False,False,False,False,False,False,True,False,...,False,False,False,False,False,True,False,False,False,False
342602,1,6d4e29bb,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
342601,1,6d4e29bb,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False


In [177]:
Ft_viewed_product = Ft_viewed_product.groupby(["mes","person"]).sum().reset_index()
Ft_viewed_product.to_csv('FT_viewed_product_train.csv',index = False)

## Features sobre brand listing

In [178]:
converted[converted['event'] == "brand listing"].head(1)

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,channel,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version,label
39,2018-05-14 23:50:44,brand listing,ad93850f,NaN,NaN,NaN,NaN,NaN,NaN,"5904,6248,2697,6023,6250,6011,5907,2696,333",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [179]:
#Por el momento no uso el evento, solo contiene skus

## Features sobre ad campaign hit

In [180]:
#Este evento contiene la plataforma de campania desde la cual el usuario accede
Ft_campaign_source = converted[converted["event"] == "ad campaign hit"][["timestamp","person","campaign_source"]]

In [181]:
Ft_campaign_source["mes"] = converted["timestamp"].dt.month

In [182]:
Ft_campaign_source.drop(columns = "timestamp",inplace = True)

In [183]:
Ft_campaign_source =  Ft_campaign_source.sort_values(by = "mes")

In [184]:
Ft_campaign_source = change_column_order(Ft_campaign_source,"mes",0)

In [185]:
top_Campaigns = Ft_campaign_source["campaign_source"].value_counts().nlargest(10).index

In [186]:
for campaign in top_Campaigns:
    Ft_campaign_source[campaign] = (Ft_campaign_source['campaign_source'] == campaign).astype(bool)

In [187]:
Ft_campaign_source.drop(columns ="campaign_source",inplace=True)

In [188]:
Ft_campaign_source = Ft_campaign_source.groupby(["mes","person"]).sum().reset_index()
Ft_campaign_source.to_csv('Ft_campaign_source_train.csv',index = False)

## Features sobre search engine

In [189]:
#Este evento contiene la buscador desde el cual el user accede
Ft_search_engine = converted[converted["event"] == "search engine hit"][["timestamp","person","search_engine"]]

In [190]:
Ft_search_engine["mes"] = converted["timestamp"].dt.month
Ft_search_engine.drop(columns = "timestamp",inplace = True)
Ft_search_engine =  Ft_search_engine.sort_values(by = "mes")
Ft_search_engine = change_column_order(Ft_search_engine,"mes",0)
Ft_search_engine =  Ft_search_engine.sort_values(by = "mes")
top_search_engines = Ft_search_engine["search_engine"].value_counts().index

In [191]:
for engine in top_search_engines:
    Ft_search_engine[engine] = (Ft_search_engine['search_engine'] == engine).astype(bool)

In [192]:
Ft_search_engine.drop(columns ="search_engine",inplace=True)

In [193]:
Ft_search_engine = Ft_search_engine.groupby(["mes","person"]).sum().reset_index()
Ft_search_engine.to_csv('Ft_search_engine_train.csv',index = False)

## Features sobre checkout

In [194]:
Ft_checkout = converted[converted["event"] == "checkout"].iloc[:,:9].drop(columns = ["sku","url"])

In [195]:
Ft_checkout["mes"] = Ft_checkout["timestamp"].dt.month
Ft_checkout.drop(columns = "timestamp",inplace = True)

In [196]:
Ft_checkout  = change_column_order(Ft_checkout,"mes",0)

In [197]:
Ft_checkout = Ft_checkout.sort_values(by="mes").drop(columns = "event")


In [198]:
modelos_populares = Ft_checkout["model"].value_counts().nlargest(15).index
modelos_populares = pd.Series(modelos_populares).values
modelos_populares 

array(['iPhone 5s', 'iPhone 6', 'iPhone 6S', 'Samsung Galaxy J5',
       'Samsung Galaxy S7', 'Samsung Galaxy J7 Prime', 'iPhone 7 Plus',
       'iPhone 7', 'Motorola Moto G2 3G Dual', 'Samsung Galaxy S8',
       'Samsung Galaxy S7 Edge', 'Samsung Galaxy S6 Flat',
       'Motorola Moto G4 Plus', 'iPhone 6 Plus', 'iPhone 6S Plus'],
      dtype=object)

In [199]:
for model in modelos_populares:
    Ft_checkout[model] = (Ft_checkout['model'] == model).astype(bool)

In [200]:
Ft_checkout.drop(columns = ["model"],inplace = True)

In [201]:
modelos_populares = pd.Series(modelos_populares).apply(lambda x: x +"_checkouts").values
Ft_checkout.rename(columns=dict(zip(Ft_checkout.iloc[:,5:], modelos_populares)),inplace=True)

In [202]:
#colores
coloresPopulares = Ft_checkout["color"].value_counts().nlargest(15).index
coloresPopulares = pd.Series(coloresPopulares).values
coloresPopulares 

array(['Preto', 'Dourado', 'Branco', 'Cinza espacial', 'Prateado',
       'Ouro Rosa', 'Rosa', 'Prata', 'Preto Matte', 'Platinum', 'Azul',
       'Ametista', 'Cinza', 'Ouro', 'Preto Brilhante'], dtype=object)

In [203]:
for color in coloresPopulares:
    Ft_checkout[color] = (Ft_checkout['color'] == color).astype(bool)

In [204]:
Ft_checkout.drop(columns = ["color"],inplace = True)

In [205]:
coloresPopulares = pd.Series(coloresPopulares).apply(lambda x: x +"_checkouts").values
Ft_checkout.rename(columns=dict(zip(Ft_checkout.iloc[:,19:], coloresPopulares)),inplace=True)

In [206]:
#condiciones
for condition in condiciones:
    Ft_checkout[condition] = (Ft_checkout['condition'] == condition).astype(bool)

In [207]:
conditionChecks = pd.Series(condiciones).apply(lambda x: x +"_checkouts").values
Ft_checkout.rename(columns=dict(zip(Ft_checkout.iloc[:,34:], conditionChecks)),inplace=True)

In [208]:
Ft_checkout.drop(columns = ["condition","storage"],inplace = True)

In [209]:
Ft_checkout = Ft_checkout.groupby(["mes","person"]).sum().reset_index()
Ft_checkout.to_csv('Ft_checkout_train.csv',index = False)

## Features sobre conversion

In [210]:
Ft_conversion = converted[converted["event"] == "conversion"].iloc[:,:9].drop(columns = ["sku","url"])

In [211]:
Ft_conversion["mes"] = Ft_conversion["timestamp"].dt.month
Ft_conversion.drop(columns = "timestamp",inplace = True)

In [212]:
Ft_conversion  = change_column_order(Ft_conversion,"mes",0)

In [213]:
Ft_conversion = Ft_conversion.sort_values(by="mes").drop(columns = "event")

In [214]:
modelos_populares = Ft_conversion["model"].value_counts().nlargest(15).index
modelos_populares = pd.Series(modelos_populares).values
modelos_populares 

array(['iPhone 5s', 'Samsung Galaxy J5', 'iPhone 6',
       'Motorola Moto G2 3G Dual', 'iPhone 6S', 'Samsung Galaxy J7 Prime',
       'Motorola Moto G4 Plus', 'Samsung Galaxy S7',
       'Samsung Galaxy S6 Flat', 'iPhone 4S', 'Samsung Galaxy S7 Edge',
       'Samsung Galaxy S5', 'Samsung Galaxy J2 Prime TV',
       'Samsung Galaxy Gran Prime Duos TV', 'Samsung Galaxy S6 Edge'],
      dtype=object)

In [215]:
for model in modelos_populares:
    Ft_conversion[model] = (Ft_conversion['model'] == model).astype(bool)

In [216]:
Ft_conversion.drop(columns = ["model"],inplace = True)

In [217]:
modelos_populares = pd.Series(modelos_populares).apply(lambda x: x +"_conversiones").values
Ft_conversion.rename(columns=dict(zip(Ft_conversion.iloc[:,5:], modelos_populares)),inplace=True)


In [218]:
for color in coloresPopulares:
    Ft_conversion[color] = (Ft_conversion['color'] == color).astype(bool)

In [219]:
Ft_conversion.drop(columns = ["color"],inplace = True)

In [220]:
coloresPopulares = pd.Series(coloresPopulares).apply(lambda x: x +"_conversiones").values
Ft_conversion.rename(columns=dict(zip(Ft_conversion.iloc[:,19:], coloresPopulares)),inplace=True)

In [221]:
#condiciones
for condition in condiciones:
    Ft_conversion[condition] = (Ft_conversion['condition'] == condition).astype(bool)

In [222]:
conditionConv = pd.Series(condiciones).apply(lambda x: x +"_conversiones").values
Ft_conversion.rename(columns=dict(zip(Ft_conversion.iloc[:,34:], conditionConv)),inplace=True)

In [223]:
Ft_conversion.drop(columns = ["condition","storage"],inplace = True)

In [224]:
Ft_conversion = Ft_conversion.groupby(["mes","person"]).sum().reset_index()
Ft_conversion.to_csv('Ft_conversion_train.csv',index = False)

## Feature sobre cantidad de eventos por user

In [10]:
converted["mes"] = converted["timestamp"].dt.month
Ft_cant_eventos = converted.groupby(["person","mes"])["event"].size()
Ft_cant_eventos = pd.DataFrame(data = Ft_cant_eventos)\
                    .rename(columns = {'event' : 'cantidad_eventos'}).reset_index().sort_values(by="mes")

In [11]:
Ft_cant_eventos.to_csv("Ft_cant_eventos_train.csv")

# TEST

In [4]:
usuarios = pd.merge(events,trocafone,on="person",how="inner")
usuarios["timestamp"] = pd.to_datetime(usuarios["timestamp"])

## Features sobre visited site

In [16]:
Ft_visited_site = pd.concat([usuarios[usuarios["event"] == "visited site"].iloc[:,:3],\
                         usuarios[usuarios["event"] == "visited site"].iloc[:,14:]],axis =1)

In [17]:
popular_countries = Ft_visited_site["country"].value_counts().nlargest(10).index

for country in popular_countries:
    Ft_visited_site[country] = (Ft_visited_site['country'] == country).astype(bool)
    
Ft_visited_site.drop(columns = "country",inplace = True)

Ft_visited_site = Ft_visited_site[Ft_visited_site["channel"] != "Unknown"]

Ft_visited_site["channel"].value_counts()

channels = Ft_visited_site["channel"].value_counts().index

for channel in channels:
    Ft_visited_site[channel] = (Ft_visited_site['channel'] == channel).astype(bool)

Ft_visited_site = Ft_visited_site.drop(columns = "channel")

popular_browsers = Ft_visited_site["browser_version"].value_counts().nlargest(10).index

for browser in popular_browsers:
    Ft_visited_site[browser] = (Ft_visited_site['browser_version'] == channel).astype(bool)

Ft_visited_site = Ft_visited_site.drop(columns = "browser_version")

recurrencias = Ft_visited_site["new_vs_returning"].value_counts().index

for recurrencia in recurrencias:
    Ft_visited_site[recurrencia] = (Ft_visited_site['new_vs_returning'] == recurrencia).astype(bool)
    
Ft_visited_site = Ft_visited_site.drop(columns = ["city","region","new_vs_returning"])

populars_resolutions = Ft_visited_site["screen_resolution"].value_counts().nlargest(10).index

for resolution in populars_resolutions:
    Ft_visited_site[resolution] = (Ft_visited_site['screen_resolution'] == resolution).astype(bool)
    
Ft_visited_site = Ft_visited_site.drop(columns = ["screen_resolution","operating_system_version"])


tipoDisp = Ft_visited_site["device_type"].value_counts().index

for dispositivo in tipoDisp:
    Ft_visited_site[dispositivo] = (Ft_visited_site['device_type'] == dispositivo).astype(bool)
    
Ft_visited_site = Ft_visited_site.drop(columns = ["device_type","event"])\
                    .rename(columns = {"Unknown" : "Unknown_country"})

Ft_visited_site["mes"] = Ft_visited_site["timestamp"].dt.month
Ft_visited_site.drop(columns = "timestamp", inplace =True)

def change_column_order(df, col_name, index):
    cols = df.columns.tolist()
    cols.remove(col_name)
    cols.insert(index, col_name)
    return df[cols]

Ft_visited_site = change_column_order(Ft_visited_site,"mes",0)

Ft_visited_site = Ft_visited_site.sort_values(by="mes")

Ft_visited_site = Ft_visited_site.groupby(["mes","person"]).sum().reset_index()

Ft_visited_site.to_csv('Ft_visited_site_test.csv',index = False)


## Features sobre view products

In [18]:
Ft_viewed_product = usuarios[usuarios["event"] == "viewed product"].iloc[:,0:9]\
                    .drop(columns = ["event","url"])

Ft_viewed_product["mes"] = Ft_viewed_product["timestamp"].dt.month

Ft_viewed_product.drop(columns = "timestamp",inplace = True)

Ft_viewed_product = change_column_order(Ft_viewed_product,"mes",0)

Ft_viewed_product = Ft_viewed_product.sort_values(by="mes")

modelos_populares = Ft_viewed_product["model"].value_counts().nlargest(15).index

modelos_populares = pd.Series(modelos_populares).values

for modelo in modelos_populares:
    Ft_viewed_product[modelo] = (Ft_viewed_product['model'] == modelo).astype(bool)

modelosView = pd.Series(modelos_populares).apply(lambda x: x +"_view").values
Ft_viewed_product.rename(columns=dict(zip(Ft_viewed_product.iloc[:,7:], modelosView)),inplace=True)

Ft_viewed_product.drop(columns = "model",inplace = True)

#15 mas vistos
colores = Ft_viewed_product["color"].value_counts().nlargest(15).index

for color in colores:
    Ft_viewed_product[color] = (Ft_viewed_product['color'] == color).astype(bool)
    
Ft_viewed_product.drop(columns = "color",inplace = True)

Ft_viewed_product.drop(columns = "sku",inplace = True)

coloresView = pd.Series(colores).apply(lambda x: x +"_view").values
Ft_viewed_product.rename(columns=dict(zip(Ft_viewed_product.iloc[:,19:], coloresView)),inplace=True)

Ft_viewed_product.drop(columns = "storage",inplace = True)

condiciones = Ft_viewed_product.condition.value_counts().index

for condition in condiciones:
    Ft_viewed_product[condition] = (Ft_viewed_product['condition'] == condition).astype(bool)
    
conditionView = pd.Series(condiciones).apply(lambda x: x +"_view").values
Ft_viewed_product.rename(columns=dict(zip(Ft_viewed_product.iloc[:,33:], conditionView)),inplace=True)

Ft_viewed_product.drop(columns = "condition",inplace = True)

Ft_viewed_product = Ft_viewed_product.groupby(["mes","person"]).sum().reset_index()
Ft_viewed_product.to_csv('FT_viewed_product_test.csv',index = False)


## Features sobre ad campaign hit

In [19]:
Ft_campaign_source = usuarios[usuarios["event"] == "ad campaign hit"][["timestamp","person","campaign_source"]]

Ft_campaign_source["mes"] = usuarios["timestamp"].dt.month

Ft_campaign_source.drop(columns = "timestamp",inplace = True)

Ft_campaign_source =  Ft_campaign_source.sort_values(by = "mes")

Ft_campaign_source = change_column_order(Ft_campaign_source,"mes",0)

top_Campaigns = Ft_campaign_source["campaign_source"].value_counts().nlargest(10).index

for campaign in top_Campaigns:
    Ft_campaign_source[campaign] = (Ft_campaign_source['campaign_source'] == campaign).astype(bool)
    
Ft_campaign_source.drop(columns ="campaign_source",inplace=True)

Ft_campaign_source = Ft_campaign_source.groupby(["mes","person"]).sum().reset_index()
Ft_campaign_source.to_csv('Ft_campaign_source_test.csv',index = False)

## Features sobre search engine hit

In [20]:
Ft_search_engine = usuarios[usuarios["event"] == "search engine hit"][["timestamp","person","search_engine"]]

Ft_search_engine["mes"] = usuarios["timestamp"].dt.month
Ft_search_engine.drop(columns = "timestamp",inplace = True)
Ft_search_engine =  Ft_search_engine.sort_values(by = "mes")
Ft_search_engine = change_column_order(Ft_search_engine,"mes",0)
Ft_search_engine =  Ft_search_engine.sort_values(by = "mes")
top_search_engines = Ft_search_engine["search_engine"].value_counts().index

for engine in top_search_engines:
    Ft_search_engine[engine] = (Ft_search_engine['search_engine'] == engine).astype(bool)
    
Ft_search_engine.drop(columns ="search_engine",inplace=True)

Ft_search_engine = Ft_search_engine.groupby(["mes","person"]).sum().reset_index()
Ft_search_engine.to_csv('Ft_search_engine_test.csv',index = False)


## Features sobre checkout

In [21]:
Ft_checkout = usuarios[usuarios["event"] == "checkout"].iloc[:,:9].drop(columns = ["sku","url"])

Ft_checkout["mes"] = Ft_checkout["timestamp"].dt.month

Ft_checkout.drop(columns = "timestamp",inplace = True)

Ft_checkout  = change_column_order(Ft_checkout,"mes",0)

Ft_checkout = Ft_checkout.sort_values(by="mes").drop(columns = "event")

modelos_populares = Ft_checkout["model"].value_counts().nlargest(15).index
modelos_populares = pd.Series(modelos_populares).values

for model in modelos_populares:
    Ft_checkout[model] = (Ft_checkout['model'] == model).astype(bool)
    
Ft_checkout.drop(columns = ["model"],inplace = True)

modelos_populares = pd.Series(modelos_populares).apply(lambda x: x +"_checkouts").values
Ft_checkout.rename(columns=dict(zip(Ft_checkout.iloc[:,5:], modelos_populares)),inplace=True)

#colores
coloresPopulares = Ft_checkout["color"].value_counts().nlargest(15).index
coloresPopulares = pd.Series(coloresPopulares).values

for color in coloresPopulares:
    Ft_checkout[color] = (Ft_checkout['color'] == color).astype(bool)
    
Ft_checkout.drop(columns = ["color"],inplace = True)

coloresPopulares = pd.Series(coloresPopulares).apply(lambda x: x +"_checkouts").values
Ft_checkout.rename(columns=dict(zip(Ft_checkout.iloc[:,19:], coloresPopulares)),inplace=True)

#condiciones
for condition in condiciones:
    Ft_checkout[condition] = (Ft_checkout['condition'] == condition).astype(bool)
    
conditionChecks = pd.Series(condiciones).apply(lambda x: x +"_checkouts").values
Ft_checkout.rename(columns=dict(zip(Ft_checkout.iloc[:,34:], conditionChecks)),inplace=True)

Ft_checkout.drop(columns = ["condition","storage"],inplace = True)

Ft_checkout = Ft_checkout.groupby(["mes","person"]).sum().reset_index()
Ft_checkout.to_csv('Ft_checkout_test.csv',index = False)

## Features sobre conversion

In [22]:
Ft_conversion = usuarios[usuarios["event"] == "conversion"].iloc[:,:9].drop(columns = ["sku","url"])

Ft_conversion["mes"] = Ft_conversion["timestamp"].dt.month

Ft_conversion.drop(columns = "timestamp",inplace = True)

Ft_conversion  = change_column_order(Ft_conversion,"mes",0)

Ft_conversion = Ft_conversion.sort_values(by="mes").drop(columns = "event")

modelos_populares = Ft_conversion["model"].value_counts().nlargest(15).index
modelos_populares = pd.Series(modelos_populares).values

for model in modelos_populares:
    Ft_conversion[model] = (Ft_conversion['model'] == model).astype(bool)
    
Ft_conversion.drop(columns = ["model"],inplace = True)

modelos_populares = pd.Series(modelos_populares).apply(lambda x: x +"_conversiones").values
Ft_conversion.rename(columns=dict(zip(Ft_conversion.iloc[:,5:], modelos_populares)),inplace=True)

for color in coloresPopulares:
    Ft_conversion[color] = (Ft_conversion['color'] == color).astype(bool)
    
Ft_conversion.drop(columns = ["color"],inplace = True)

#condiciones
for condition in condiciones:
    Ft_conversion[condition] = (Ft_conversion['condition'] == condition).astype(bool)
    
conditionConv = pd.Series(condiciones).apply(lambda x: x +"_conversiones").values
Ft_conversion.rename(columns=dict(zip(Ft_conversion.iloc[:,34:], conditionConv)),inplace=True)

Ft_conversion.drop(columns = ["condition","storage"],inplace = True)

Ft_conversion = Ft_conversion.groupby(["mes","person"]).sum().reset_index()
Ft_conversion.to_csv('Ft_conversion_test.csv',index = False)

### Feature sobre cantidad de eventos por user

In [6]:
usuarios["mes"] = usuarios["timestamp"].dt.month
Ft_cant_eventos = usuarios.groupby(["mes","person"])["event"].size()
Ft_cant_eventos = pd.DataFrame(data = Ft_cant_eventos)\
                    .rename(columns = {'event' : 'cantidad_eventos'}).reset_index()
Ft_cant_eventos.to_csv("Ft_cant_eventos_test.csv")